<a href="https://colab.research.google.com/github/saxena-ak/chatbot_NN/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing all the important packages
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import io
import pickle
from google.colab import files
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
#downloading "punkt" tokenizer that creates tokens from text
nltk.download('punkt')
#downloding lexical database for English
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
#initializing important variables
word=[]
classes = []
doc = []

#to upload JSON file that contains data to Google Colab
uploaded = files.upload()
data = next(iter(uploaded.values()))
intent_data = json.loads(data.decode())

Saving intents.json to intents.json


In [4]:
#Just to see how data looks like inside the file
i = intent_data['intents']
intent_data['intents'][0]["patterns"]

['Hi there',
 'How are you',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day']

In [5]:
# Converting text into tokens and storing all classes variable
for intent in intent_data['intents']:
  for pattern in intent['patterns']:
    #First we create a token for each word present in the patterns of intent_data
    token = nltk.word_tokenize(pattern)
    #Add created tokens to word variable
    word.extend(token)
    #add tokens and respective tags to documents in the corpus
    doc.append((token, intent['tag']))
    # add unique tags to classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


In [6]:
# Second we lemmaztize each word, convert them into lower alphabet and remove all the duplicates
non_word = ['?', '!']
word = [lemmatizer.lemmatize(w.lower()) for w in word if w not in non_word]
word = sorted(list(set(word)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(doc), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(word), "unique lemmatized words", word)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
# creating training data to train the model
train = []
# creating empty array to store output
output = [0] * len(classes)
# creating training set and bag of words for each sentence
for d in doc:
    # initialize the bag of words
    bag = []
    # list of tokenized words for the pattern
    token_word = d[0]
    # creating lemma for each word, converting them to lower case 
    token_word = [lemmatizer.lemmatize(word.lower()) for word in token_word]
    # Adding 1 to bag of words if a word match is found, 0 if not
    for w in word:
        bag.append(1) if w in token_word else bag.append(0)
    
    output_row = list(output)
    output_row[classes.index(d[1])] = 1
    
    #creating Training Data
    train.append([bag, output_row])

In [8]:
# shuffing train data and cpnverting it into np.array
random.shuffle(train)
train = np.array(train)

# creating train sets to train the model
train_x = list(train[:,0])
train_y = list(train[:,1])
print("X:", train_x)
print("y:",train_y)

X: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Creating NN model with three layers. 
# 1st - 128 neurons
# 2nd - 64 neurons 
# 3rd - output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
# Compiling the model. Using SGD with Nesterov accelerated gradient. Using Cross Entropy as loss
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

print("model created")

Epoch 1/200
10/10 [==============================] - 1s 2ms/step - loss: 2.1579 - accuracy: 0.1996
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1389 - accuracy: 0.1694
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1214 - accuracy: 0.1248
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9133 - accuracy: 0.4558
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8854 - accuracy: 0.4366
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6818 - accuracy: 0.5225
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4905 - accuracy: 0.5690
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4539 - accuracy: 0.7172
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3187 - accuracy: 0.7624
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1845 - accuracy: 0.7591
Epoch 11/

In [11]:
#This function will preprocess the user input and predict the Bot's response using the model 
def get_result(msg):

  #tokenizing the message
  resp_token = nltk.word_tokenize(msg)
  resp_token = [lemmatizer.lemmatize(w.lower()) for w in resp_token]

  #creating bag of words for the message tokens 
  bag = [0]*len(word)
  for s in resp_token:
    for i,w in enumerate(word):
      if(w == s):
        bag[i] = 1

  x_pred = np.array(bag)

  #predicting the response using model
  response = model.predict(np.array([x_pred]))[0]

  threshold = 0.25
  res = [[i,r] for i,r in enumerate(response) if r > threshold]
  res.sort(key = lambda x:x[1],reverse = True)
  
  res_lst = []
  for r in res:
    res_lst.append({"intent":classes[r[0]],"probability": str(r[1])})
  
  #choosing the result from the outcome
  tag = res_lst[0]['intent']
  intent_list = intent_data['intents']
  for i in intent_list:
    if(i['tag'] == tag):
      result = random.choice(i['responses'])
      break
  return result

In [14]:
#using Google Colab Widgits to see the outcome
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

interact(get_result, msg= "");

interactive(children=(Text(value='', description='msg'), Output()), _dom_classes=('widget-interact',))